### aim: 

### date: 

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [3]:
import numpy as np
from cmocean import cm
import cartopy as cp
import cartopy.crs as ccrs
import netCDF4 as nc
import matplotlib.pyplot as plt
import xarray as xr
import sys
sys.path.append('/gpfs/home/mep22dku/scratch/SOZONE')
#list of models
sys.path.append('/gpfs/home/mep22dku/scratch/SOZONE/UTILS')
import lom
import utils as ut

%matplotlib inline
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')
import cartopy.feature as cfeature
from importlib import reload
import matplotlib.path as mpath
import glob
import pickle
import pandas as pd
import seawater
import time
plt.rcParams.update({'font.size': 12})
font = {'family' : 'normal',
'weight' : 'normal',
'size'   : 12}

plt.rc('font', **font)

In [4]:
scendict = {
    '1A': {
        'hist_str': 'bc370',
        'fut_str': 'be682',
        'name': 'HIST.OZONE \n LOW TEMP.',
        'name2':'1A: NatlOzone-SSP126',
        'color':'#E8D215',
        'color2':'orange'},
    '1B': {
        'hist_str': 'bc370',
        'fut_str': 'ce417',
        'name': 'HIST. OZONE \n HIGH TEMP.',
        'name2':'1B: NatlOzone-SSP370',
        'color':'#87800A',
        'color2':'orangered'},
    '2A': {
        'hist_str': 'cj198',
        'fut_str': 'cj880',
        'name': 'FIXED OZONE \n LOW TEMP.',
        'name2':'2A: Ozone1950-SSP126',
        'color':'#2DC18E',
        'color2':'mediumseagreen'},
    '2B': {
        'hist_str': 'cj198',
        'fut_str': 'cj881',
        'name': 'FIXED OZONE \n HIGH TEMP.',
        'name2':'2B: Ozone1950-SSP370',
        'color':'#18765C',
        'color2':'green'},
    '3A': {
        'hist_str': 'cj200',
        'fut_str': 'cj484',
        'name': '1990 OZONE \n LOW TEMP.',
        'name2':'3A: Ozone1990-SSP126',
        'color':'#FF462B',
        'color2':'dodgerblue'},
    '3B': {
        'hist_str': 'cj200',
        'fut_str': 'cj504',
        'name': '1990 OZONE \n HIGH TEMP.',
        'name2':'3B: Ozone1990-SSP370',
        'color':'#822722',
        'color2':'mediumblue'}
}


ukmesh = xr.open_dataset('/gpfs/data/greenocean/software/resources/MEDUSA/mesh_mask_eORCA1_wrk.nc')
ukmesh['area'] = ukmesh.tmask[0,:,:] * ukmesh.e1t[:,:] * ukmesh.e2t[:,:]


def make_yearlist_ukesm(yrst, yren, tscen, dtype = 'grid-T'):
    print(f'SCENARIO {tscen}')
    dslist = []

    for y in range(yrst,yren):
        if ((y<1990) & ((tscen == '3A') | (tscen == '3B'))):
            tstr = scendict['1A']['hist_str']
        elif y<2015:
            tstr = scendict[tscen]['hist_str']

        else:
            tstr = scendict[tscen]['fut_str']

        try:
            td = glob.glob(f'/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/*{tstr}*{y}*{dtype}*')
            dslist.append(td[0])
        except:
            pass
            #print(f'FAIL {tscen},{tstr}, {y}')
    return dslist

tdir = '/gpfs/home/mep22dku/scratch/SOZONE/windAnalyis/paperFigures/intermediateCalculations/'

yst = 1950; yen = 2100




In [5]:
tylist = make_yearlist_ukesm(yst,yen,'1A', dtype = 'TS')
tylist

SCENARIO 1A


['/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1950_grid-T-TS.nc',
 '/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1951_grid-T-TS.nc',
 '/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1952_grid-T-TS.nc',
 '/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1953_grid-T-TS.nc',
 '/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1954_grid-T-TS.nc',
 '/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1955_grid-T-TS.nc',
 '/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1956_grid-T-TS.nc',
 '/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1957_grid-T-TS.nc',
 '/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1958_grid-T-TS.nc',
 '/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1959_grid-T-TS.nc',
 '/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1960_grid-T-TS.nc',
 '/gpfs/da

In [20]:
tsc = '1A'
tylist = make_yearlist_ukesm(yst,yen,tsc, dtype = 'TS')

for y in range(yst, yen):
    
    print(y)
    print(tylist[y-1950])
    print()
    mld_1A = xr.open_dataset(tylist[y-1950])
    
    v  = mld_1A.votemper.isel(deptht=0).sel(y=slice(0,114)).weighted(ukmesh['area'].sel(y=slice(0,114))).mean(dim = ['x','y'])
    v.name = "votemper"
    v.attrs["units"] = "m"
    v.attrs["desc"] = "area-weighted mean south of -50 from ukesm"
    v.to_netcdf(f'./y{y}_-50_SST_{tsc}.nc')
    

SCENARIO 1A
1950
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1950_grid-T-TS.nc

1951
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1951_grid-T-TS.nc

1952
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1952_grid-T-TS.nc

1953
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1953_grid-T-TS.nc

1954
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1954_grid-T-TS.nc

1955
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1955_grid-T-TS.nc

1956
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1956_grid-T-TS.nc

1957
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1957_grid-T-TS.nc

1958
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1958_grid-T-TS.nc

1959
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1959_grid-T-TS.nc

1960
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc37

In [21]:
tsc = '1B'
tylist = make_yearlist_ukesm(yst,yen,tsc, dtype = 'TS')

for y in range(yst, yen):
    
    print(y)
    print(tylist[y-1950])
    print()
    mld_1A = xr.open_dataset(tylist[y-1950])
    
    v  = mld_1A.votemper.isel(deptht=0).sel(y=slice(0,114)).weighted(ukmesh['area'].sel(y=slice(0,114))).mean(dim = ['x','y'])
    v.name = "votemper"
    v.attrs["units"] = "m"
    v.attrs["desc"] = "area-weighted mean south of -50 from ukesm"
    v.to_netcdf(f'./y{y}_-50_SST_{tsc}.nc')
    
tsc = '2A'
tylist = make_yearlist_ukesm(yst,yen,tsc, dtype = 'TS')

for y in range(yst, yen):
    
    print(y)
    print(tylist[y-1950])
    print()
    mld_1A = xr.open_dataset(tylist[y-1950])
    
    v  = mld_1A.votemper.isel(deptht=0).sel(y=slice(0,114)).weighted(ukmesh['area'].sel(y=slice(0,114))).mean(dim = ['x','y'])
    v.name = "votemper"
    v.attrs["units"] = "m"
    v.attrs["desc"] = "area-weighted mean south of -50 from ukesm"
    v.to_netcdf(f'./y{y}_-50_SST_{tsc}.nc')
    
tsc = '2B'
tylist = make_yearlist_ukesm(yst,yen,tsc, dtype = 'TS')

for y in range(yst, yen):
    
    print(y)
    print(tylist[y-1950])
    print()
    mld_1A = xr.open_dataset(tylist[y-1950])
    
    v  = mld_1A.votemper.isel(deptht=0).sel(y=slice(0,114)).weighted(ukmesh['area'].sel(y=slice(0,114))).mean(dim = ['x','y'])
    v.name = "votemper"
    v.attrs["units"] = "m"
    v.attrs["desc"] = "area-weighted mean south of -50 from ukesm"
    v.to_netcdf(f'./y{y}_-50_SST_{tsc}.nc')
    
    
tsc = '3A'
tylist = make_yearlist_ukesm(yst,yen,tsc, dtype = 'TS')

for y in range(yst, yen):
    
    print(y)
    print(tylist[y-1950])
    print()
    mld_1A = xr.open_dataset(tylist[y-1950])
    
    v  = mld_1A.votemper.isel(deptht=0).sel(y=slice(0,114)).weighted(ukmesh['area'].sel(y=slice(0,114))).mean(dim = ['x','y'])
    v.name = "votemper"
    v.attrs["units"] = "m"
    v.attrs["desc"] = "area-weighted mean south of -50 from ukesm"
    v.to_netcdf(f'./y{y}_-50_SST_{tsc}.nc')
    
    
tsc = '3B'
tylist = make_yearlist_ukesm(yst,yen,tsc, dtype = 'TS')

for y in range(yst, yen):
    
    print(y)
    print(tylist[y-1950])
    print()
    mld_1A = xr.open_dataset(tylist[y-1950])
    
    v  = mld_1A.votemper.isel(deptht=0).sel(y=slice(0,114)).weighted(ukmesh['area'].sel(y=slice(0,114))).mean(dim = ['x','y'])
    v.name = "votemper"
    v.attrs["units"] = "m"
    v.attrs["desc"] = "area-weighted mean south of -50 from ukesm"
    v.to_netcdf(f'./y{y}_-50_SST_{tsc}.nc')
    

    

SCENARIO 1B
1950
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1950_grid-T-TS.nc

1951
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1951_grid-T-TS.nc

1952
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1952_grid-T-TS.nc

1953
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1953_grid-T-TS.nc

1954
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1954_grid-T-TS.nc

1955
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1955_grid-T-TS.nc

1956
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1956_grid-T-TS.nc

1957
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1957_grid-T-TS.nc

1958
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1958_grid-T-TS.nc

1959
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1959_grid-T-TS.nc

1960
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc37

HDF5-DIAG: Error detected in HDF5 (1.10.4) thread 47019694291072:
  #000: H5F.c line 444 in H5Fcreate(): unable to create file
    major: File accessibilty
    minor: Unable to open file
  #001: H5Fint.c line 1364 in H5F__create(): unable to open file
    major: File accessibilty
    minor: Unable to open file
  #002: H5Fint.c line 1615 in H5F_open(): unable to lock the file
    major: File accessibilty
    minor: Unable to open file
  #003: H5FD.c line 1640 in H5FD_lock(): driver lock request failed
    major: Virtual File Layer
    minor: Can't update object
  #004: H5FDsec2.c line 941 in H5FD_sec2_lock(): unable to lock file, errno = 11, error message = 'Resource temporarily unavailable'
    major: File accessibilty
    minor: Bad file ID accessed


PermissionError: [Errno 13] Permission denied: b'/gpfs/scratch/mep22dku/SOZONE/windAnalyis/paperFigures/intermediateCalculations/y2081_-50_SST_1A.nc'

In [22]:
tsc = '1A'
tylist = make_yearlist_ukesm(yst,yen,tsc, dtype = 'TS')

for y in range(yst, yen):
    
    print(y)
    print(tylist[y-1950])
    print()
    mld_1A = xr.open_dataset(tylist[y-1950])
    
    v  = mld_1A.votemper.isel(deptht=0).sel(y=slice(0,114)).weighted(ukmesh['area'].sel(y=slice(0,114))).mean(dim = ['x','y'])
    v.name = "votemper"
    v.attrs["units"] = "m"
    v.attrs["desc"] = "area-weighted mean south of -50 from ukesm"
    v.to_netcdf(f'./y{y}_-50_SST_{tsc}.nc')
    

SCENARIO 1A
1950
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1950_grid-T-TS.nc

1951
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1951_grid-T-TS.nc

1952
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1952_grid-T-TS.nc

1953
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1953_grid-T-TS.nc

1954
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1954_grid-T-TS.nc

1955
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1955_grid-T-TS.nc

1956
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1956_grid-T-TS.nc

1957
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1957_grid-T-TS.nc

1958
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1958_grid-T-TS.nc

1959
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc370_1y_1959_grid-T-TS.nc

1960
/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/nemo_bc37

In [14]:
y*_SST_1A*

<xarray.Dataset>
Dimensions:       (time_counter: 12)
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2091-01-31 ... 2091-12-31
    deptht        float32 ...
Data variables:
    votemper      (time_counter) float64 ...